In [1]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset, ConcatDataset
from torch.optim import Adam
from torch import nn
import numpy as np
from tqdm.auto import tqdm

code block for running in google colab

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
if torch.cuda.is_available():
  DEVICE = 'cuda'
else:
  DEVICE = 'cpu'

cuda


In [4]:
TRANSFORM = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor()
    ])

DOMAIN_EPOCHS = 60
PRED_EPOCHS = 60
BATCH_SIZE = 16

In [5]:
class MyModel(nn.Module):
    def __init__(self, train_mode: bool=False, domain: bool=False, num_classes: int=10):
        super().__init__()
        self.prediction_model = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', weights=None)
        self.prediction_model.classifier[1] = nn.Linear(self.prediction_model.last_channel, num_classes)
        self.prediction_model.to(DEVICE)

        self.domain_model = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', weights=None)
        self.domain_model.classifier[1] = nn.Linear(self.domain_model.last_channel, 1)
        self.domain_model.to(DEVICE)

        self.train_mode = train_mode
        self.domain = domain

    def forward(self, img):
        # training prediction flow
        if self.train_mode:
            if self.domain:
                prob = self.domain_model(img)
                prob = prob.view(-1, 1)
                return prob
            else:
                return self.prediction_model(img)

        # normal prediction flow
        logit = self.domain_model(img)
        logit = logit.view(1, -1)

        prob = torch.sigmoid(logit)
        predictions = self.prediction_model(img)

        for i, p in enumerate(prob.flatten()):
            if p < 0.5:
                artificial_prediction = torch.tensor([0] * 10)
                rand_num = np.random.randint(0, 10)
                artificial_prediction[rand_num] = 1
                predictions[i] = artificial_prediction

        return predictions


In [6]:
class BinaryDataLoader(Dataset):
    def __init__(self, dataset, label):
        self.dataset = dataset
        self.label = label

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        item, _ = self.dataset[index]
        label = self.label
        return item, label

In [7]:
def get_domain_data(in_path, out_path):
    in_data = datasets.ImageFolder(in_path, transform=TRANSFORM)
    out_data = datasets.ImageFolder(out_path, transform=TRANSFORM)

    in_data_loader = BinaryDataLoader(in_data, 1)
    out_data_loader = BinaryDataLoader(out_data, 0)

    combined_dataset = ConcatDataset([in_data_loader, out_data_loader])
    combined_data_loader = DataLoader(combined_dataset, batch_size=BATCH_SIZE, shuffle=True)

    return combined_data_loader

In [8]:
def learn(path_to_in_domain: str, path_to_out_domain: str):
    in_data = datasets.ImageFolder(path_to_in_domain, transform=TRANSFORM)
    in_data_loader = DataLoader(in_data, batch_size=BATCH_SIZE, shuffle=True)

    comb = get_domain_data(path_to_in_domain, path_to_out_domain)

    model = MyModel()

    # train domain
    model.train_mode = True
    model.domain = True

    optimizer = Adam(model.parameters(), lr=1e-4)
    criterion = nn.BCEWithLogitsLoss()

    for e in range(DOMAIN_EPOCHS):
        print(f'epoch: {e}')
        running_loss = 0.
        for i, data in tqdm(enumerate(comb), total=len(comb)):
            inputs, labels = data
            inputs = inputs.to(DEVICE)

            labels = labels.view(-1, 1)
            labels = labels.to(DEVICE)

            optimizer.zero_grad()
            outputs = model(inputs)

            loss = criterion(outputs.float(), labels.float())
            running_loss += loss.item()
            loss.backward()

            optimizer.step()


        print(f'running loss for epoch {e}: {running_loss}')


    # train prediction
    model.train_mode = True
    model.domain = False

    optimizer = Adam(model.parameters(), lr=1e-4)
    criterion = nn.CrossEntropyLoss()

    for e in range(PRED_EPOCHS):
        print(f'epoch: {e}')
        running_loss = 0.
        for i, data in tqdm(enumerate(in_data_loader), total=len(in_data_loader)):
            inputs, labels = data
            inputs = inputs.to(DEVICE)
            labels = labels.to(DEVICE)

            optimizer.zero_grad()
            outputs = model(inputs)

            loss = criterion(outputs, labels)
            running_loss += loss.item()
            loss.backward()

            optimizer.step()
        print(f'running loss for epoch {e}: {running_loss}')
    torch.save(model.state_dict(), 'myModel.pth')
    return model

In [9]:
def accuracy(path_to_eval_folder: str, model) -> float:
    data = datasets.ImageFolder(path_to_eval_folder, transform=TRANSFORM)
    data_loader = DataLoader(data, batch_size=BATCH_SIZE, shuffle=True)

    model.train_mode = False
    model.eval()

    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in tqdm(data_loader, total=len(data_loader)):
            inputs = inputs.to(DEVICE)
            labels = labels.to(DEVICE)

            outputs = model(inputs)
            outputs = outputs.view(-1, 10)
            outputs = outputs.to(DEVICE)

            _, predicted = torch.max(outputs, dim=1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return correct/total

In [10]:
# model = MyModel()
model = learn('drive/MyDrive/A4data/in-domain-train', 'drive/MyDrive/A4data/out-domain-train')

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


epoch: 0


Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


  0%|          | 0/76 [00:00<?, ?it/s]

running loss for epoch 0: 51.71790951490402
epoch: 1


...

running loss for epoch 57: 7.342795591801405
epoch: 58


  0%|          | 0/43 [00:00<?, ?it/s]

running loss for epoch 58: 11.066071739420295
epoch: 59


  0%|          | 0/43 [00:00<?, ?it/s]

running loss for epoch 59: 6.096360605210066


In [11]:
new_model = MyModel()
new_model.to(DEVICE)
state_dict = torch.load('myModel.pth', map_location=DEVICE)
new_model.load_state_dict(state_dict)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


<All keys matched successfully>

In [12]:
def test_domain_model(model, in_dom, out_dom):
    comb = get_domain_data(in_dom, out_dom)

    model.train_mode = True
    model.domain = True
    model.eval()

    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in tqdm(comb, total=len(comb)):
            inputs = inputs.to(DEVICE)
            labels = labels.to(DEVICE)
            labels = labels.view(-1, 1)

            logits = model(inputs)
            logits = logits.view(-1, 1)
            probs = torch.sigmoid(logits)

            preds = (probs > 0.5).long()

            total += labels.size(0)
            correct += (preds == labels).sum().item()

    return correct/total

In [13]:
exp = test_domain_model(new_model, 'drive/MyDrive/A4data/in-domain-eval', 'drive/MyDrive/A4data/out-domain-eval')
print(exp)

  0%|          | 0/76 [00:00<?, ?it/s]

0.6496290189612531


In [14]:
def test_prediction_model(model, in_dom):
    data = datasets.ImageFolder(in_dom, transform=TRANSFORM)
    data_loader = DataLoader(data, batch_size=BATCH_SIZE, shuffle=True)

    model.train_mode = True
    model.domain = False
    model.eval()

    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in tqdm(data_loader, total=len(data_loader)):
            inputs = inputs.to(DEVICE)
            labels = labels.to(DEVICE)

            outputs = model(inputs)
            outputs = outputs.view(-1, 10)
            outputs = outputs.to(DEVICE)

            _, predicted = torch.max(outputs, dim=1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return correct/total

In [15]:
exp = test_prediction_model(new_model, 'drive/MyDrive/A4data/in-domain-eval')
print(exp)

  0%|          | 0/43 [00:00<?, ?it/s]

0.46131386861313867


In [16]:
acc = accuracy('drive/MyDrive/A4data/in-domain-eval', new_model)
print(acc)
acc_2 = accuracy('drive/MyDrive/A4data/out-domain-eval', new_model)
print(acc_2)

  0%|          | 0/43 [00:00<?, ?it/s]

tensor(0.3156, device='cuda:0')
tensor(1.0000, device='cuda:0')
tensor(0.0283, device='cuda:0')
tensor(1.0000, device='cuda:0')
tensor(0.9168, device='cuda:0')
tensor(0.9971, device='cuda:0')
tensor(0.9038, device='cuda:0')
tensor(0.8693, device='cuda:0')
tensor(0.9810, device='cuda:0')
tensor(0.3077, device='cuda:0')
tensor(0.0226, device='cuda:0')
tensor(0.9626, device='cuda:0')
tensor(0.9831, device='cuda:0')
tensor(0.6863, device='cuda:0')
tensor(0.9997, device='cuda:0')
tensor(0.9803, device='cuda:0')
tensor(0.9998, device='cuda:0')
tensor(0.1215, device='cuda:0')
tensor(1.0000, device='cuda:0')
tensor(0.1340, device='cuda:0')
tensor(0.5586, device='cuda:0')
tensor(0.9981, device='cuda:0')
tensor(0.9987, device='cuda:0')
tensor(0.9869, device='cuda:0')
tensor(0.9058, device='cuda:0')
tensor(0.7384, device='cuda:0')
tensor(0.9050, device='cuda:0')
tensor(1., device='cuda:0')
tensor(0.8262, device='cuda:0')
tensor(0.8952, device='cuda:0')
tensor(0.7838, device='cuda:0')
tensor(0.984

  0%|          | 0/33 [00:00<?, ?it/s]

tensor(0.0105, device='cuda:0')
tensor(0.0546, device='cuda:0')
tensor(0.0130, device='cuda:0')
tensor(0.9785, device='cuda:0')
tensor(0.8970, device='cuda:0')
tensor(0.9666, device='cuda:0')
tensor(0.9999, device='cuda:0')
tensor(0.0104, device='cuda:0')
tensor(0.2491, device='cuda:0')
tensor(0.0003, device='cuda:0')
tensor(0.0785, device='cuda:0')
tensor(0.0710, device='cuda:0')
tensor(0.2331, device='cuda:0')
tensor(0.1695, device='cuda:0')
tensor(0.3617, device='cuda:0')
tensor(0.9917, device='cuda:0')
tensor(0.1047, device='cuda:0')
tensor(0.9837, device='cuda:0')
tensor(1.0000, device='cuda:0')
tensor(0.0161, device='cuda:0')
tensor(0.9985, device='cuda:0')
tensor(0.9760, device='cuda:0')
tensor(0.9302, device='cuda:0')
tensor(0.0018, device='cuda:0')
tensor(0.0398, device='cuda:0')
tensor(0.4652, device='cuda:0')
tensor(0.9667, device='cuda:0')
tensor(0.7616, device='cuda:0')
tensor(0.8384, device='cuda:0')
tensor(0.9987, device='cuda:0')
tensor(0.8615, device='cuda:0')
tensor(0